In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, pacf
from statsmodels.tsa.ar_model import AutoReg
from typing import Tuple

In [5]:
def load_data(file_path: str) -> pd.DataFrame:
    """Įkelia elektros suvartojimo duomenis."""
    df: pd.DataFrame = pd.read_csv(file_path, parse_dates=['Date'], index_col='Date')
    return df

def check_stationarity(series: pd.Series) -> Tuple[bool, float]:
    """Tikrina, ar laiko eilutė yra stacionari naudojant ADF testą."""
    result = adfuller(series)
    p_value: float = result[1]
    return p_value < 0.05, p_value  # True, jei stacionari

def plot_acf_pacf(series: pd.Series) -> None:
    """Nubraižo autokoreliacijos ir dalinės autokoreliacijos grafikus."""
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    sm.graphics.tsa.plot_acf(series, lags=20, ax=axes[0])
    sm.graphics.tsa.plot_pacf(series, lags=20, ax=axes[1])
    axes[0].set_title("Autokoreliacijos funkcija (ACF)")
    axes[1].set_title("Dalinės autokoreliacijos funkcija (PACF)")
    plt.show()

def train_ar_model(series: pd.Series, lag: int) -> AutoReg:
    """Treniruoja autoregresinį (AR) modelį."""
    model = AutoReg(series, lags=lag).fit()
    return model

def forecast_ar(model: AutoReg, steps: int) -> np.ndarray:
    """Atlieka ateities reikšmių prognozę naudojant AR modelį."""
    predictions: np.ndarray = model.predict(start=len(model.model.endog), end=len(model.model.endog) + steps - 1)
    return predictions

In [ ]:
# === PAGRINDINĖ PROGRAMOS DALIS ===
if __name__ == "__main__":
    # 1. Duomenų įkėlimas
    file_path: str = "electricity_data.csv"  # Nurodykite teisingą failo pavadinimą
    data: pd.DataFrame = load_data(file_path)
    
    # 2. Tikriname stacionarumą
    is_stationary, p_value = check_stationarity(data['KWH_mean'])
    print(f"Stacionarumas: {is_stationary}, p-reikšmė: {p_value:.5f}")
    
    # 3. Jei reikia, diferencijuojame duomenis
    if not is_stationary:
        data['KWH_mean'] = data['KWH_mean'].diff().dropna()
    
    # 4. Parodome ACF ir PACF grafikus
    plot_acf_pacf(data['KWH_mean'].dropna())
    
    # 5. Treniravimas (naudojame PACF rezultatą pasirinkti lag reikšmę)
    lag: int = 3  # Galima keisti pagal PACF analizę
    ar_model: AutoReg = train_ar_model(data['KWH_mean'].dropna(), lag)
    
    # 6. Prognozė
    steps: int = 12  # Prognozuojame 12 mėnesių į priekį
    forecast_values: np.ndarray = forecast_ar(ar_model, steps)
    
    # 7. Rezultatų vizualizacija
    plt.figure(figsize=(10, 5))
    plt.plot(data.index, data['KWH_mean'], label='Istoriniai duomenys')
    plt.plot(pd.date_range(start=data.index[-1], periods=steps, freq='M'), forecast_values, label='Prognozė', linestyle='dashed')
    plt.legend()
    plt.xlabel("Laikas")
    plt.ylabel("KWH_mean")
    plt.title("Autoregresijos modelio prognozė")
    plt.show()

In [ ]:
class Analyzer():
  def __init__(self):
  self.DF_DATA = None
 
  def fit(self, data, X_features:list, y_target:str):
    self.DF_DATA = data[X_features+[y_target]].groupby(X_features).mean().reset_index()
    pass
 
  def predict(self, d:str):
   dt = pd.to_datetime(d)
   M = dt.month
   D = dt.day_of_week
   h24 = self.DF_DATA.query(expr='MM == @M and DD == @D')
   return h24[['HH', 'value']]

# [['MM','DD', 'HH', 'value']]

Kita užduotis:
https://github.com/Call-for-Code/Spot-Challenge-Wildfires/blob/main/data/Readme_Docs_Wildfires-Datasets_2020-11.pdf